In [18]:
import random
from dataclasses import dataclass

In [19]:
@dataclass
class Battery:
    soh: float = 100.0  # State of Health
    age: float = 0.0  # Age in years
    total_km: float = 0.0  # Total kilometers traveled

In [20]:
class BatteryLifecycleSimulation:
    def __init__(self):
        self.battery = Battery()
        self.current_life = 1
        self.years_in_current_life = 0
        self.battery_capacity = 250  # kWh
        self.initial_cost = self.battery_capacity * 140  # USD
        self.refurbish_cost_1 = self.battery_capacity * 30  # USD
        self.refurbish_cost_2 = self.battery_capacity * 20  # USD
        self.recycling_value = 1300  # USD
        self.total_cost = self.initial_cost - self.recycling_value  # Reduce upfront cost by recycling value

    def degrade_battery(self, years, km_per_year, soh_loss_per_year):
        for _ in range(years):
            self.battery.soh -= soh_loss_per_year + random.uniform(-1, 1)  # Add some randomness
            self.battery.age += 1
            self.battery.total_km += km_per_year
            self.years_in_current_life += 1
            if self.battery.soh <= self.get_end_of_life_soh():
                return False
        return True

    def get_end_of_life_soh(self):
        if self.current_life == 1:
            return 80
        elif self.current_life == 2:
            return 70
        else:
            return 50

    def refurbish(self):
        if self.current_life == 1:
            self.battery.soh = 90
            self.total_cost += self.refurbish_cost_1
        elif self.current_life == 2:
            self.battery.soh = 75
            self.total_cost += self.refurbish_cost_2

    def calculate_lease_rental(self, years_in_life, cost):
        return cost / years_in_life

    def run_simulation(self):
        # First life: Battery Electric Truck
        truck_km_per_year = 72000
        truck_soh_loss_per_year = 5
        while self.degrade_battery(1, truck_km_per_year, truck_soh_loss_per_year):
            pass
        
        first_life_years = self.years_in_current_life
        first_life_lease = self.calculate_lease_rental(first_life_years, self.initial_cost - self.recycling_value)

        self.refurbish()
        self.current_life += 1
        self.years_in_current_life = 0

        # Second life: Electric 2-wheeler
        two_wheeler_km_per_year = 15000
        two_wheeler_soh_loss_per_year = 4
        while self.degrade_battery(1, two_wheeler_km_per_year, two_wheeler_soh_loss_per_year):
            pass

        second_life_years = self.years_in_current_life
        second_life_lease = self.calculate_lease_rental(second_life_years, self.refurbish_cost_1)

        self.refurbish()
        self.current_life += 1
        self.years_in_current_life = 0

        # Third life: Stationary storage
        stationary_soh_loss_per_year = 3
        while self.degrade_battery(1, 0, stationary_soh_loss_per_year):
            pass

        third_life_years = self.years_in_current_life
        third_life_lease = self.calculate_lease_rental(third_life_years, self.refurbish_cost_2)

        return {
            "total_years": self.battery.age,
            "truck_km": self.battery.total_km - (second_life_years * two_wheeler_km_per_year),
            "two_wheeler_km": second_life_years * two_wheeler_km_per_year,
            "stationary_years": third_life_years,
            "first_life_lease": first_life_lease,
            "second_life_lease": second_life_lease,
            "third_life_lease": third_life_lease,
            "total_cost": self.total_cost,
            "effective_cost": self.total_cost - self.recycling_value  # Consider recycling value in effective cost
        }


In [21]:
num_simulations = 1000
results = []

In [22]:
for _ in range(num_simulations):
    sim = BatteryLifecycleSimulation()
    results.append(sim.run_simulation())

In [23]:
# Calculate averages
avg_total_years = sum(r["total_years"] for r in results) / num_simulations
avg_truck_km = sum(r["truck_km"] for r in results) / num_simulations
avg_two_wheeler_km = sum(r["two_wheeler_km"] for r in results) / num_simulations
avg_stationary_years = sum(r["stationary_years"] for r in results) / num_simulations
avg_first_life_lease = sum(r["first_life_lease"] for r in results) / num_simulations
avg_second_life_lease = sum(r["second_life_lease"] for r in results) / num_simulations
avg_third_life_lease = sum(r["third_life_lease"] for r in results) / num_simulations
avg_total_cost = sum(r["total_cost"] for r in results) / num_simulations
avg_effective_cost = sum(r["effective_cost"] for r in results) / num_simulations

In [24]:
print(f"Average total battery lifetime: {avg_total_years:.2f} years")
print(f"Average distance traveled by truck: {avg_truck_km:.2f} km")
print(f"Average distance traveled by two-wheeler: {avg_two_wheeler_km:.2f} km")
print(f"Average years in stationary storage: {avg_stationary_years:.2f} years")
print(f"Average yearly lease rental for first life: ${avg_first_life_lease:.2f}")
print(f"Average yearly lease rental for second life: ${avg_second_life_lease:.2f}")
print(f"Average yearly lease rental for third life: ${avg_third_life_lease:.2f}")
print(f"Average total cost over lifetime: ${avg_total_cost:.2f}")
print(f"Average effective cost (considering recycling value): ${avg_effective_cost:.2f}")

Average total battery lifetime: 18.89 years
Average distance traveled by truck: 322848.00 km
Average distance traveled by two-wheeler: 82920.00 km
Average years in stationary storage: 8.88 years
Average yearly lease rental for first life: $7609.46
Average yearly lease rental for second life: $1368.14
Average yearly lease rental for third life: $566.07
Average total cost over lifetime: $46200.00
Average effective cost (considering recycling value): $44900.00
